# 3.1 Imports

In [7]:
# import libraries
import numpy as np
import pandas as pd

import urllib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

import time
from PIL import Image

In [8]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [26]:
# create function
# open web browser

def configure_driver():
    # incognito window
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    
    # add the argument and make the browser Headless.
    # chrome_options.add_argument("--headless")

    # open web browser
    driver = webdriver.Chrome('C:/Users/Leung/Desktop/chromedriver.exe', options=chrome_options)
    return driver

In [ ]:
# adding multiple-threading into the function:

# no. of threads to use
# threads = min(MAX_THREADS, len(all_links))

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(detail_scraping_function, all_links)

In [3]:
# load data
df = pd.read_csv('data/activewear.csv')

In [4]:
df.head()

,name,price,img_file,url,num_price
0,LACE LIGHTWEAR JACKET,HKD 903.00,image/LACE LIGHTWEAR JACKET.png,https://www.calvinklein.com/hk/en/lace-lightwe...,903.0
1,LOGO SHORT SLEEVE TEE,HKD 390.00,image/LOGO SHORT SLEEVE TEE.png,https://www.calvinklein.com/hk/en/logo-short-s...,390.0
2,LACE FITSENSE MESH HEM LEGGINGS,HKD 553.00,image/LACE FITSENSE MESH HEM LEGGINGS.png,https://www.calvinklein.com/hk/en/lace-fitsens...,553.0
3,BACK V LACE SHORT SLEEVE DRESS,HKD 623.00,image/BACK V LACE SHORT SLEEVE DRESS.png,https://www.calvinklein.com/hk/en/back-v-lace-...,623.0
4,CK ONE TANGA BIKINI,HKD 392.00,image/CK ONE TANGA BIKINI.png,https://www.calvinklein.com/hk/en/ck-one-tanga...,392.0


In [52]:
# testing
n = 0

name = df['name'].iloc[n]
price = df['price'].iloc[n]
url = df['url'].iloc[n]

In [6]:
name

'LACE LIGHTWEAR JACKET'

In [53]:
url

'https://www.calvinklein.com/hk/en/lace-lightwear-jacket-4WT1O553.html?dwvar_4WT1O553_color=BRIGHT%20WHITE'

In [36]:
# open web browser
driver = configure_driver()

In [37]:
# search engine
search_engine = 'https://www.amazon.com/?currency=HKD&language=en_US'

In [40]:
# input website
driver.get(search_engine)

In [41]:
# may need time sleep
time.sleep(0.5)

In [42]:
# select option
option = driver.find_element_by_id('icp-touch-link-cop')
option.click()

In [44]:
# select currency setting
currency = driver.find_element_by_xpath('//span[@class="a-button-inner"]')
currency.click()

In [45]:
# select hkd as currency
hkd = driver.find_element_by_xpath('//li[@class="a-dropdown-item"]/a[contains(text(), "Hong Kong Dollar")]')
hkd.click()

In [46]:
# save change
save = driver.find_element_by_id('icp-btn-save')
save.click()

In [50]:
# select shopping department
department = driver.find_element_by_xpath('//select[@class="nav-search-dropdown searchSelect nav-progressive-attrubute nav-progressive-search-dropdown"]/option[contains(text(), "Women\'s Fashion")]')
department.click()

In [51]:
# input search item
search_bar = driver.find_element_by_id('twotabsearchtextbox')
search_bar.send_keys(name)
search_bar.send_keys(Keys.ENTER)

In [17]:
# get product image

# get all images
images = driver.find_elements_by_xpath('//div[@class="a-section aok-relative s-image-square-aspect"]')

In [20]:
len(images)

57

In [21]:
# get product name
names = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none a-spacing-top-small"]')

In [22]:
len(names)

109

In [24]:
for i,j in enumerate(names):
    print(i)
    print(j.text)

0
Columbia 哥倫比亞 男款 Watertight II 外套
1
US$59
43
2
Amazon Essentials Men's Water-Resistant Softshell Jacket
3
US$35
90
4
Columbia 男款 Ascender Softshell 前拉鍊夾克
5
US$67
91
6
Weatherproof 男款中等重量防水防風軟殼夾克（尺寸 S-3XL）
7
US$55
17
8
Amazon Essentials Men's Lightweight Water-Resistant Packable Hooded Puffer Jacket
9
US$46
70
10
Amazon Essentials Men's Lightweight Water-Resistant Packable Puffer Jacket
11
US$39
00
12
Columbia 哥倫比亞 男款 Steens Mountain 全拉鍊 2.0 刷毛夾克
13
US$34
99
US$39.99
14
Loungewear 女款運動服套裝 - 2 件式長袖套頭衫 + 抽繩運動褲 運動服飾組
15
Wantdo 女款 輕量 羽絨 夾克 可收納 蓬鬆外套 連帽保暖夾克
16
US$58
97
儲存 10% 使用優惠卷 （一些尺寸／顏色）
17
Wantdo 男款棉質立領輕量前拉鍊夾克
18
US$57
97
US$61.57
儲存 5% 使用優惠卷 （一些尺寸／顏色）
19
WenVen 男款休閒水洗棉質軍裝夾克
20
US$55
99
儲存 5% 使用優惠卷 （一些尺寸／顏色）
21
WenVen 男款 秋季休閒外套 輕量 棉質 軍事 連帽夾克
22
US$52
99
儲存 10% 使用優惠卷 （一些尺寸／顏色）
23
Wantdo 男款 軍用 棉質 輕盈 夾克 休閒 立領外套
24
US$52
97
US$61.57
儲存 10% 使用優惠卷 （一些尺寸／顏色）
25
WenVen 男士休閒帆布棉質軍裝翻領夾克
26
US$56
99
儲存 5% 使用優惠卷 （一些尺寸／顏色）
27
TBMPOY 男款 防風 飛行員夾克 輕量 跑步 風衣 戶外 高爾夫 時尚 外套
28
US$34
99
29
WenVen 男款軍裝夾克 輕量 棉

In [ ]:
# testing

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()



In [ ]:
# close web browser
driver.close()

In [ ]:
# call the final function as 'main'

In [ ]:
# check if target items displayed by WebFriverWait()
# to save time
try:
    WebDriverWait(driver, 5).until(lambda s: s.find_element_by_id("search-results-category-target").is_displayed())
except TimeoutException:
    return None

In [ ]:
# threading lock
# add to the end of the function that is not thread-safe
import threading
csv_write_lock = threading.Lock()

# only one thread allowed
with csv_write_lock:
    with open('data/file_name.csv', mode='a') as f:
        text_writer = csv.writer(f, delimiter=',', quotchar='"', quoting=csv.QUOTE_MINIMAL)
        text_writer.writerow([name, price, img_file, url])

In [ ]:
# END